In [206]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt


In [207]:
transform = transforms.Compose([transforms.Resize((64, 64)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])
dataset = ImageFolder(
    root="C:/Users/com/Downloads/datasets/train",
    transform=transform,
)
loader = DataLoader(dataset, batch_size=64, shuffle=True)


In [208]:



class Generator(nn.Module):
   def __init__(self):
       super(Generator, self).__init__()
      
       # 생성자를 구성하는 층 정의
       self.gen = nn.Sequential(
           nn.ConvTranspose2d(100, 512, kernel_size=4, bias=False),
           nn.BatchNorm2d(512),
           nn.ReLU(),

           nn.ConvTranspose2d(512, 256, kernel_size=4, 
                              stride=2, padding=1, bias=False),
           nn.BatchNorm2d(256),
           nn.ReLU(),

           nn.ConvTranspose2d(256, 128, kernel_size=4, 
                              stride=2, padding=1, bias=False),
           nn.BatchNorm2d(128),
           nn.ReLU(),

           nn.ConvTranspose2d(128, 64, kernel_size=4, 
                              stride=2, padding=1, bias=False),
           nn.BatchNorm2d(64),
           nn.ReLU(),

           nn.ConvTranspose2d(64, 3, kernel_size=4, 
                              stride=2, padding=1, bias=False),
           nn.Tanh()
       )

   def forward(self, x):
       return self.gen(x)

In [209]:
class Discriminator(nn.Module):
   def __init__(self):
       super(Discriminator, self).__init__()
      
       # 감별자를 구성하는 층의 정의
       self.disc = nn.Sequential(
           nn.Conv2d(3, 64, kernel_size=4, 
                     stride=2, padding=1, bias=False),
           nn.BatchNorm2d(64),
           nn.LeakyReLU(0.2),

           nn.Conv2d(64, 128, kernel_size=4, 
                     stride=2, padding=1, bias=False),
           nn.BatchNorm2d(128),
           nn.LeakyReLU(0.2),

           nn.Conv2d(128, 256, kernel_size=4, 
                     stride=2, padding=1, bias=False),
           nn.BatchNorm2d(256),
           nn.LeakyReLU(0.2),

           nn.Conv2d(256, 512, kernel_size=4, 
                     stride=2, padding=1, bias=False),
           nn.BatchNorm2d(512),
           nn.LeakyReLU(0.2),

           nn.Conv2d(512, 1, kernel_size=4),
           nn.Sigmoid()
       )

   def forward(self, x):
       return self.disc(x)

In [210]:
def weights_init(m):
    # 층의 종류 추출
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        # 합성곱층 초기화
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        # 배치정규화층 초기화
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [211]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 생성자 정의
G = Generator().to(device)
# ❶ 생성자 가중치 초기화
G.apply(weights_init)

# 감별자 정의
D = Discriminator().to(device)
# ❷ 감별자 가중치 초기화
D.apply(weights_init)

import tqdm

from torch.optim.adam import Adam

D_optim = Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
G_optim = Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [212]:
for epochs in range(1):
    iterator = tqdm.tqdm(enumerate(loader, 0), total=len(loader))
    for i, data in iterator:
        D_optim.zero_grad()

        real = D(data[0].to(device))
        label = torch.ones_like(data[1], dtype=torch.float32).to(device)
        Dloss_real = nn.BCEWithLogitsLoss()(torch.squeeze(real), label)
        Dloss_real.backward(retain_graph=True)  # retain_graph=True
        noise = torch.randn(label.shape[0], 100, 1, 1, device=device)
        fake = G(noise)
        output = D(fake.detach())
        label_fake = torch.zeros_like(data[1], dtype=torch.float32).to(device)
        Dloss_fake = nn.BCEWithLogitsLoss()(torch.squeeze(output), label_fake)
        Dloss_fake.backward()

        D_optim.step()

        # 생성자 학습
        G_optim.zero_grad()
        output = D(fake)
        Gloss = nn.BCEWithLogitsLoss()(torch.squeeze(output), label)
        Gloss.backward()
        G_optim.step()

        iterator.set_description(f"epoch:{epochs} iteration:{i} D_loss:{Dloss_real + Dloss_fake} G_loss:{Gloss}")

    # 모델 저장
    torch.save(G.state_dict(), "Generator.pth")
    torch.save(D.state_dict(), "Discriminator.pth")

epoch:0 iteration:2 D_loss:1.3324689865112305 G_loss:0.6917083859443665:   2%|▏         | 3/155 [00:07<06:14,  2.47s/it]


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
   G.load_state_dict(
       torch.load("./Generator.pth", map_location=device))
   # 특징 공간 상의 랜덤한 하나의 점을 지정
   feature_vector = torch.randn(1, 100, 1, 1).to(device)
   # 이미지 생성
   pred = G(feature_vector).squeeze()
   pred = pred.permute(1, 2, 0).cpu().numpy()
   plt.imshow(pred)
   plt.title("predicted image")
   plt.show()